In [1]:
from bs4 import BeautifulSoup as bs
import requests as r
import pandas as pd
import numpy as np
import datetime
import csv
import os
import sys
from state_cleaner import *
from selenium import webdriver
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'C:\\Users\\Noah\\CovidMobile'

In [3]:
def check_save(file_path):
    path = ('data/{}'.format(file_path))
    if os.path.exists(path):
        os.remove(path)
    else:
        print("{} does not exist".format(file_path))
    
    
def write_out(file_path, url):
    path = ('data/{}'.format(file_path))
    print(file_path)
    with open(path, mode='w', newline='') as file:
        file_writer = csv.writer(file, delimiter=',')
        print(url)
        with r.get(url, verify=False, stream=True) as res:
            lines = (line.decode('utf-8',errors='ignore') for line in res.iter_lines())
            for row in csv.reader(lines):

                file_writer.writerow(row)

                
                
def create_stuff(file_path,url):
    check_save(file_path)
    write_out(file_path,url)

### Apple Data

In [4]:
status = 0
attempt = 0
while status != 200:
    appl_day = str(datetime.datetime.now().date() - datetime.timedelta(attempt))
    url = "https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-{}.csv".format(appl_day)
    print(url)
    status = r.get(url, verify=False, stream=True).status_code
    attempt = attempt + 1
    print(status,attempt)

https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-17.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


404 1
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-16.csv
404 2
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-15.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


404 3
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-14.csv
404 4
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-13.csv
200 5


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [5]:
apple_file = 'apple_data_{}.csv'.format(appl_day)
create_stuff(apple_file, url)

apple_data_2020-04-13.csv
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-13.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


## Census Data


In [6]:
start = 'https://web.archive.org/web/20150807220054/http://quickfacts.census.gov/qfd/download_data.html'

In [7]:
res = r.get(start, verify=False)
soup = bs(res.content,'html.parser')
sys.path.insert(0, "/notebooks")

options = webdriver.ChromeOptions()

driver = webdriver.Chrome()
driver.get(start)



for link in soup.findAll('a', href=True):
    if '.txt' in link['href']:
        url = link['href']
        print(link)
        driver.find_element_by_xpath('//a[@href="'+link['href']+'"]').click()
        url = driver.find_element_by_id("playback").get_attribute("src")
        create_stuff('{}_{}'.format(link['href'].rsplit('/')[-1],str(datetime.datetime.now().date())),url)
        driver.execute_script("window.history.go(-1)")


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<a href="download/DataSet.txt">DataSet.txt</a>
DataSet.txt_2020-04-17 does not exist
DataSet.txt_2020-04-17
https://web.archive.org/web/20150821182814if_/http://quickfacts.census.gov:80/qfd/download/DataSet.txt


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<a href="download/DataDict.txt">DataDict.txt</a>
DataDict.txt_2020-04-17 does not exist
DataDict.txt_2020-04-17
https://web.archive.org/web/20150821061818if_/http://quickfacts.census.gov:80/qfd/download/DataDict.txt


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


...

C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<a href="download/FootNotes.txt">FootNotes.txt</a>
FootNotes.txt_2020-04-17 does not exist
FootNotes.txt_2020-04-17
https://web.archive.org/web/20150905124937if_/http://quickfacts.census.gov/qfd/download/FootNotes.txt


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<a href="download/FootText.txt">FootText.txt</a>
FootText.txt_2020-04-17 does not exist
FootText.txt_2020-04-17
https://web.archive.org/web/20150717163825if_/http://quickfacts.census.gov:80/qfd/download/FootText.txt


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


# New York Times - Covid Cases

In [8]:
nyt = 'https://github.com/nytimes/covid-19-data'
res = r.get(nyt)
soup = bs(res.content,'html.parser')
for hr in soup.findAll('a',href=True):
    if 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/' in hr['href']:
        create_stuff('{}_{}'.format(hr['href'].rsplit('/')[-1],str(datetime.datetime.now().date())),hr['href'])
        print(hr['href'])

us.csv_2020-04-17
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv
us-states.csv_2020-04-17
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv
us-counties.csv_2020-04-17
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv
us.csv_2020-04-17
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv
us-states.csv_2020-04-17
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv
us-counties.csv_2020-04-17
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv


# Neighboring county map

In [15]:
url = 'https://www2.census.gov/geo/docs/reference/county_adjacency.txt'
df = pd.read_csv(url,delimiter='\t',encoding='ISO-8859-1',header=None)
df = df.ffill()
df.columns = ['county','FIPS','nearby_county','nearby_FIPS'] 
df.to_csv('county_adjacency',index=False)

# Updated Census Data

In [9]:
df_county_names = pd.read_csv('data/FIPS_CountyName.txt_2020-04-16', sep='delimiter', header=None)
df_county_names = df_county_names[0].str.split(' ',n=1,expand=True)
df_county_names = pd.concat([df_county_names[0],df_county_names[1].str.split(',',n=1,expand=True)],axis=1)
df_county_names.columns = ['FIPS','County','State']
df_county_names['State'] = df_county_names['State'].str.replace(' ', '')
df_county_names.shape,df_county_names.head()
df_county_names['state'] = df_county_names['State'].apply(replace_acronym)

C:\Users\Noah\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
df_county_names

In [ ]:
df_county_names['strng'] = df_county_names['County'].str.replace(' ','').str.lower() + df_county_names['state'].str.replace(' ','').str.lower()

In [ ]:
key_file = open("census_api.txt")
lines = key_file.readlines()
census_api_key = lines[0].rstrip()

In [22]:
import pandas as pd

#allows us to store results of API call cleanly
import json

'''#get list of all zipcodes in Los Angeles County separated by commas
laZips = open('laZips.txt', 'r').readlines()
laZips = [z.replace('\n', '') for z in laZips]
laZips = ','.join(laZips)'''

#put your census API key here
apiKey = census_api_key

#construct the API call we will use
url = 'https://api.census.gov/data/2017/acs/acs1?get=NAME,B02015_009E,B02015_009M&for=county:*&key={}'.format(census_api_key)
#call the API and collect the response
response = r.get(url)

In [8]:
import pandas as pd

#allows us to store results of API call cleanly
import json

'''#get list of all zipcodes in Los Angeles County separated by commas
laZips = open('laZips.txt', 'r').readlines()
laZips = [z.replace('\n', '') for z in laZips]
laZips = ','.join(laZips)'''

#put your census API key here
apiKey = census_api_key

#construct the API call we will use
url = 'https://api.census.gov/data/2017/acs/acs1?get=NAME,B02015_009E,B02015_009M&for=county:*&key={}'.format(census_api_key)
#call the API and collect the response
response = requests.get(url)

#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]

#flip the order of the response from [population, zipcode] -> [zipcode, population]
formattedResponse = [item[::-1] for item in formattedResponse]

#store the response in a dataframe
laZipPopulations = pd.DataFrame(columns=['zipcode', 'population'], data=formattedResponse)

#save that dataframe to a CSV spreadsheet
laZipPopulations.to_csv('laZipPopulations.csv', index=False)

3143

In [9]:
for i in range(1,len(url)+1):
    try:
        census = 'https://www.census.gov/quickfacts/fact/table/' + ','.join(url.head(300).values) + ',US/PST045219'
        r.get(census)
        print(i)
    except:
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178


In [17]:
census = 'https://www.census.gov/quickfacts/fact/table/' + ','.join(url.tail(10).values) + ',US/PST045219'
tst = r.get(census)

In [18]:
census

'https://www.census.gov/quickfacts/fact/table/niobraracountywyoming,parkcountywyoming,plattecountywyoming,sheridancountywyoming,sublettecountywyoming,sweetwatercountywyoming,tetoncountywyoming,uintacountywyoming,washakiecountywyoming,westoncountywyoming,US/PST045219'

In [ ]:
https://www.census.gov/quickfacts/fact/table/washingtoncountycolorado,washingtoncountyalabama,US/PST045219